In [31]:
!pip3 install folium
!pip3 install wget

In [32]:
import folium
import wget
import pandas as pd

In [33]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [34]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

100% [............................................................] 7710 / 7710

In [35]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [36]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

#Initialise the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=12)

#define the circle (blue circle at NASA JSC)
nasa_circle = folium.Circle(nasa_coordinate, radius=1000, color='#001cd3', fill=True).add_child(folium.Popup('NASA Johnson Space Center')) #popup label of name

#Add the NASA circle to the map
site_map.add_child(nasa_circle)

#show the map
site_map

In [19]:
# Initialise the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. 
# In addition, add Launch site name as a popup label

for index, row in launch_sites_df.iterrows():
    
    #define the position and name of sites
    lat = row['Lat']
    long = row['Long']
    launch_site = row['Launch Site']
    
    #Define the Circle object
    circle = folium.Circle(
                #properties of circle (location, radius, color, fill)
                [lat, long],
                radius=50, 
                color='#d35400', 
                fill=True
            ).add_child(folium.Popup(launch_site)) #popup label of site name
    
    #Add the Circle objects to the map (in the loop)
    site_map.add_child(circle)
    
site_map

In [20]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [21]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    # If class=1, marker_color value will be green
    # If class=0, marker_color value will be red
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [22]:
#put launches into clusters and put onto map
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster) #note, this won't show up unless the individual launches are put onto the map (the next lines of code cover this)


for index, row in spacex_df.iterrows():
    
    #define the position and launch site for each launch
    lat = row['Lat']
    long = row['Long']
    launch_site = row['Launch Site']
    
    #deine the launch marker for each launch
    launch_marker = folium.Marker(
        [lat, long],
        # Create an icon as a text label and colour depending on launch success/failure (marker colour)
        icon=folium.Icon(color='white', icon_color=row['marker_color'])
    )
    
    # Add the launch marker for each launch to the launch clusters (not the map, the launch clusters themselves)
    marker_cluster.add_child(launch_marker)

#show the map zoomed out with the centre at NASA
site_map

In [23]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
# (This is shown in the top right hand corner)
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map


In [24]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [25]:
# find coordinate of the closet coastline
launch_site_lat = 28.56319
launch_site_long = -80.57683
launch_site_coordinates = [launch_site_lat, launch_site_long]

coastline_lat = launch_site_lat #same latitude, so the line is directly east
coastline_long = -80.56794
coastline_coordinates = [coastline_lat, coastline_long]

distance_coastline = calculate_distance(launch_site_lat, launch_site_long, coastline_lat, coastline_long)

In [26]:
print(f"The coastline is {distance_coastline:.2f} km due East from the launch site")

The coastline is 0.87 km due East from the launch site


In [27]:
#define the distance marker (which will be shown at the coastline)
distance_marker = folium.Marker(
    coastline_coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#ff5c33;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
#add the distance marker to the map and show the map
site_map.add_child(distance_marker)
site_map

In [28]:
#define the distance line (between the coastline and the launch site)
distance_line=folium.PolyLine(
    locations=[launch_site_coordinates, coastline_coordinates],
    weight=1
)

#add the distance line to the map and show the map
site_map.add_child(distance_line)
site_map

In [29]:
def distance_and_line(destination_coordinates):
    # Create a marker with distance to a closest city, railway, highway, etc.
    # Draw a line between the marker to the launch site
    
    #calculate the distance between the chosen destination and the launch site defined previously, using the calculate_distance function 
    distance_to_destination = calculate_distance(launch_site_lat, launch_site_long, destination_coordinates[0], destination_coordinates[1])
    
    
    #define the distance marker (which will be shown at the chosen destination)
    distance_marker = folium.Marker(
        destination_coordinates,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#ff5c33;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_to_destination),
            )
        )

    #define the distance line (between the chosen destination and the launch site)
    distance_line=folium.PolyLine(
        locations=[launch_site_coordinates, destination_coordinates],
        weight=1
    )

    #add the distance marker to the map 
    site_map.add_child(distance_marker)
    #add the distance line to the map 
    site_map.add_child(distance_line)

    
closest_city_coordinates = [28.10106, -80.6369]
closest_railway_coordinates = [28.57208, -80.58528]
closest_highway_coordinates = [28.56327, -80.57076]

#add distance markers and lines for closest city, railway and highway, using the above function
distance_and_line(closest_city_coordinates)
distance_and_line(closest_railway_coordinates)
distance_and_line(closest_highway_coordinates)

#show the map
site_map

In [30]:
# Import required libraries
import pandas as pd

import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output

import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

#Minimum and maximum Payload masses for the range slider
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)


#Get the launch sites from the spacex_df dataframe to use in the dropdown list
launch_sites = []
launch_sites.append({'label': 'All Sites', 'value': 'All Sites'})
for launch_site in spacex_df['Launch Site'].unique().tolist():
    launch_sites.append({'label': launch_site, 'value': launch_site})





# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),

                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                dcc.Dropdown(id='site-dropdown',
                                            options = launch_sites,
                                            value='All Sites',
                                            placeholder="Launch Site",
                                            searchable=True
                                            ),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')), #this id will be used in the callback function to change the pie chart
                                
                                html.Br(),

                                html.P("Payload range (Kg):"),

                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                                                value=[min_payload, max_payload]),




                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
            Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
    if entered_site == 'All Sites':
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='Total Success Launches by Site')
        return fig
    else:
        # return the outcomes pie chart for a selected site
        site_df = filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        fig = px.pie(site_df,values='class count',names='class',title=f'Total Success Launches for site {entered_site}')
        return fig



# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
            [Input(component_id='site-dropdown', component_property='value'), Input(component_id='payload-slider', component_property='value')]) #note the 2 inputs, so they are in a list


def get_scatter_chart(entered_site, payload_slider):
    low, high = payload_slider
    slide=(spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    dropdown_scatter=spacex_df[slide]

    #If All sites are selected, render a scatter plot to display all values for variables Payload Mass (kg) and class.
    #Point colour is set to the booster version category
    if entered_site == 'All Sites':
        fig = px.scatter(
            dropdown_scatter, x='Payload Mass (kg)', y='class',
            hover_data=['Booster Version'],
            color='Booster Version Category',
            title='Correlation between Payload and Success for all Sites')
        return fig
    else:
    #If a specific site is selected, filter the spacex_df dataframe first, then render a scatter plot to display the same as for all sites.
        dropdown_scatter = dropdown_scatter[spacex_df['Launch Site'] == entered_site]
        fig=px.scatter(
            dropdown_scatter,x='Payload Mass (kg)', y='class', 
            hover_data=['Booster Version'],
            color='Booster Version Category',
            title = f'Success by Payload Size for site {entered_site}')
        return fig

# Run the app
if __name__ == '__main__':
    app.run()
